<a href="https://colab.research.google.com/github/Antonio24ch/Proyecto_Cambio_Divisas/blob/main/Cargar_los_datos_en_Amazon_Redshift_Entregable2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Cambio de Divisas

In [1]:
!pip install sqlalchemy-redshift

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.23
    Uninstalling SQLAlchemy-2.0.23:
      Successfully uninstalled SQLAlchemy-2.0.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.50 which is incompatible.


In [2]:
#Cargando los paquetes
#Requests nos proporciona las capacidades para enviar una solicitud HTTP a un servidor.
import requests
import yaml
import json
import pytz
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text
import psycopg2

In [3]:
with open('apikey.yaml', 'r') as file:
    api_keys = yaml.safe_load(file)
api_key = api_keys['api_key']

### Obteniendo el cierre - código que se ejecutará diario

In [5]:
# Extraer la fecha actual para el horario de Mx
timezone = pytz.timezone('America/Mexico_City')
current_time = datetime.now(timezone)
formatted_date = current_time.strftime("%Y-%m-%d")

currencies_list = ['EUR', 'GBP', 'USD', 'PEN', 'BTC', 'KRW', 'INR', 'CNY', 'BRL', 'ARS', 'JPY']
currencies = ','.join(currencies_list)

base_url = f"https://api.apilayer.com/currency_data/timeframe?start_date={formatted_date}&end_date={formatted_date}&source=MXN&currencies={currencies}"

payload = {}
headers= {
  "apikey": api_key
}


response = requests.get(base_url, headers=headers, data=payload)
response.status_code

200

In [6]:
print(json.dumps(response.json(), indent=4))

{
    "success": true,
    "timeframe": true,
    "start_date": "2023-12-19",
    "end_date": "2023-12-19",
    "source": "MXN",
    "quotes": {
        "2023-12-19": {
            "MXNEUR": 0.053335,
            "MXNGBP": 0.046,
            "MXNUSD": 0.058566,
            "MXNPEN": 0.219445,
            "MXNBTC": 1.385526e-06,
            "MXNKRW": 76.03835,
            "MXNINR": 4.866344,
            "MXNCNY": 0.414506,
            "MXNBRL": 0.285097,
            "MXNARS": 47.038403,
            "MXNJPY": 8.427156
        }
    }
}


In [7]:
datos = json.loads(response.text)

Este dataframe trae únicamente una fila, la cual es el valor actual del día en curso que se estará almacenando en la tabla

In [8]:
df = pd.DataFrame(datos)
df

,success,timeframe,start_date,end_date,source,quotes
2023-12-19,True,True,2023-12-19,2023-12-19,MXN,"{'MXNEUR': 0.053335, 'MXNGBP': 0.046, 'MXNUSD'..."


### Limpiando datos

In [9]:
df_quotes = pd.json_normalize(df['quotes'])


# Combinar las nuevas columnas con el DataFrame original
df = pd.concat([df, df_quotes], axis=1).drop('quotes', axis=1)

df

,success,timeframe,start_date,end_date,source,MXNEUR,MXNGBP,MXNUSD,MXNPEN,MXNBTC,MXNKRW,MXNINR,MXNCNY,MXNBRL,MXNARS,MXNJPY
2023-12-19,True,True,2023-12-19,2023-12-19,MXN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,0.053335,0.046,0.058566,0.219445,0.000001,76.03835,4.866344,0.414506,0.285097,47.038403,8.427156


In [41]:
combined_row = df.iloc[0].combine_first(df.iloc[1])
# Convertir la serie combinada en un DataFrame de una sola fila
df_combined = pd.DataFrame([combined_row])
df_combined.rename(columns=lambda x: x.replace('MXN', '') if 'MXN' in x else x, inplace=True)
columns_to_drop = ['success', 'timeframe']
df_clean = df_combined.drop(columns=columns_to_drop).reset_index(drop=True)
df_clean

,start_date,end_date,source,EUR,GBP,USD,PEN,BTC,KRW,INR,CNY,BRL,ARS,JPY
0,2023-12-19,2023-12-19,MXN,0.053335,0.046,0.058566,0.219445,0.000001,76.03835,4.866344,0.414506,0.285097,47.038403,8.427156


In [40]:
# Conocer el tipo de dato en el df
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   start_date  1 non-null      datetime64[ns]
 1   end_date    1 non-null      datetime64[ns]
 2   source      1 non-null      object        
 3   EUR         1 non-null      float64       
 4   GBP         1 non-null      float64       
 5   USD         1 non-null      float64       
 6   PEN         1 non-null      float64       
 7   BTC         1 non-null      float64       
 8   KRW         1 non-null      float64       
 9   INR         1 non-null      float64       
 10  CNY         1 non-null      float64       
 11  BRL         1 non-null      float64       
 12  ARS         1 non-null      float64       
 13  JPY         1 non-null      float64       
dtypes: datetime64[ns](2), float64(11), object(1)
memory usage: 240.0+ bytes


### Cambiando tipo de datos

In [46]:
# Como se esta trabajando con fechas, es importante cambiar el tipo de dato de object a fecha
columnas = ['start_date', 'end_date']
df_clean[columnas] = df_clean[columnas].apply(pd.to_datetime, errors='coerce', axis=1)
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   start_date  1 non-null      datetime64[ns]
 1   end_date    1 non-null      datetime64[ns]
 2   source      1 non-null      object        
 3   EUR         1 non-null      float64       
 4   GBP         1 non-null      float64       
 5   USD         1 non-null      float64       
 6   PEN         1 non-null      float64       
 7   BTC         1 non-null      float64       
 8   KRW         1 non-null      float64       
 9   INR         1 non-null      float64       
 10  CNY         1 non-null      float64       
 11  BRL         1 non-null      float64       
 12  ARS         1 non-null      float64       
 13  JPY         1 non-null      float64       
dtypes: datetime64[ns](2), float64(11), object(1)
memory usage: 240.0+ bytes


### Enviando datos a Redshift

In [65]:
# Cargar credenciales desde el archivo YAML
from sqlalchemy import inspect
with open('credentials.yaml', 'r') as file:
    credenciales = yaml.safe_load(file)['redshift']

# Utilizar credenciales ocultas
username = credenciales['username']
password = credenciales['password']
host = credenciales['host']
port = credenciales['port']
database = credenciales['database']

# Construir la cadena de conexión
cadena_conexion = f"postgresql://{username}:{password}@{host}:{port}/{database}"

# Conectar al motor de la base de datos
engine = create_engine(cadena_conexion)
df_clean.to_sql('cambios_divisa', engine, if_exists='replace', index=False)


1

In [67]:
# Comprobar que los datos fueron cargados exitosamente en redshift al ser llamados
query = "SELECT * FROM cambios_divisa WHERE start_date::date = '2023-12-19'"
existing_data = pd.read_sql(query, engine)
existing_data

,start_date,end_date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy
0,2023-12-19,2023-12-19,MXN,0.053335,0.046,0.058566,0.219445,0.000001,76.03835,4.866344,0.414506,0.285097,47.038403,8.427156
